compte le nombre d'image que j'ai pour le glaucome

In [ ]:
from pathlib import Path
from collections import defaultdict

root = Path("data/glaucome")
exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".gif"}

# stats[dataset][label]
stats = defaultdict(lambda: defaultdict(int))
unknown = defaultdict(int)

for p in root.rglob("*"):
    if not p.is_file() or p.suffix.lower() not in exts:
        continue

    parts = [x.lower() for x in p.parts]

    # Dataset = premier dossier après data/glaucome
    try:
        dataset = p.relative_to(root).parts[0]
    except Exception:
        dataset = "UNKNOWN"

    # Label STRICTEMENT par dossier
    if "glaucoma" in parts:
        label = "glaucome"
    elif "normal" in parts:
        label = "normal"
    else:
        label = "inconnu"

    if label == "inconnu":
        unknown[dataset] += 1
    else:
        stats[dataset][label] += 1


# ----------- RAPPORT -----------
print("RAPPORT GLAUCOME (structure-based)\n")

total_g = total_n  = 0

for dataset in sorted(set(list(stats.keys()) + list(unknown.keys()))):
    g = stats[dataset].get("glaucome", 0)
    n = stats[dataset].get("normal", 0)
    u = unknown.get(dataset, 0)

    print(f"{dataset:<15} | glaucome: {g:4d} | normal: {n:4d} ")

    total_g += g
    total_n += n

print("\nTOTALS")
print("glaucome :", total_g)
print("normal   :", total_n)
print("images totales :", total_g + total_n )


compte le nombre d'image que j'ai pour le diabete

In [ ]:
from pathlib import Path
import csv

csv_path = Path("data/diabete/trainLabels.csv/trainLabels.csv")

normal_diabete = 0
diabete = 0

with csv_path.open(newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        level = int(row["level"])
        if level == 0:
            normal_diabete += 1
        else:
            diabete += 1

print("non-diabete:", normal_diabete)
print("diabete:", diabete)
print("total:", normal_diabete + diabete)


compte le nombre d'image mda

In [ ]:
from pathlib import Path

base = Path(
    "data/mda/AMDNet23/"
    "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection/"
    "AMDNet23 Dataset"
)

train_amd = sum(1 for _ in (base / "train" / "amd").iterdir())
valid_amd = sum(1 for _ in (base / "valid" / "amd").iterdir())


train_normal = sum(1 for _ in (base / "train" / "normal").iterdir())
valid_normal = sum(1 for _ in (base / "valid" / "normal").iterdir())

AMDNet23_amd = train_amd + valid_amd
AMDNet23_normal = train_normal + valid_normal

print("AMD train :", train_amd)
print("AMD valid :", valid_amd)
print("Total AMD :", AMDNet23_amd)
print("Normal train :", train_normal)
print("Normal valid :", valid_normal)
print("Total normal :", AMDNet23_normal)


In [ ]:
from pathlib import Path

base = Path("data/mda/Fundus/Fundus")

fundus_armd = sum(1 for p in (base / "ARMD").iterdir() if p.is_file())
fundus_normal = sum(1 for p in (base / "NORMAL").iterdir() if p.is_file())

print("ARMD :", fundus_armd)
print("NORMAL :", fundus_normal)
print("TOTAL :", fundus_armd + fundus_normal)


In [ ]:
from pathlib import Path

base = Path("data/mda/fundus-dataset")
image_exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}

def count_images(path):
    return sum(
        1 for p in path.iterdir()
        if p.is_file() and p.suffix.lower() in image_exts
    )

fundus_normal = count_images(base / "00. Normal")
fundus_amd = count_images(base / "14. Age Related Macular Degeneration")

print("Normal :", fundus_normal)
print("AMD (14) :", fundus_amd)
print("Total :", fundus_normal + fundus_amd)


In [ ]:
import pandas as pd
from pathlib import Path

labels_path = Path(
    "data/mda/hyamd-high-resolution-fundus-image-dataset"
    "/hyamd-high-resolution-fundus-image-dataset-for-age-related-macular-degeneration-amd-diagnosis-1.0.0"
    "/labels/labels.csv"
)

df = pd.read_csv(labels_path)

# 0 = non-malade, 1/2 = malade
hyamd_normal = (df["AMD"] == 0).sum()
hyamd_amd = (df["AMD"] > 0).sum()

print("Normal :", hyamd_normal)
print("AMD :", hyamd_amd)
print("Total :", hyamd_normal + hyamd_amd)


In [ ]:
import pandas as pd
from pathlib import Path

csv_path = Path("data/mda/ODIR-5K/full_df.csv")
df = pd.read_csv(csv_path)

ODIR_amd = (
    df["labels"]
    .astype(str)
    .str.contains(r"\bA\b", regex=True)
    .sum()
)

ODIR_normal = len(df) - ODIR_amd

print("AMD (A):", ODIR_amd)
print("Normal:", ODIR_normal)


In [ ]:
total_malade = ( AMDNet23_amd + fundus_amd + hyamd_amd + ODIR_amd)
total_normal = ( AMDNet23_normal + fundus_normal + hyamd_normal + ODIR_normal)
print("Somme malade mda :", total_malade)
print("Somme normal mda :", total_normal)


In [ ]:
print("normal:", total_n)
print("glaucoma:", total_g)

print("non-diabete:", normal_diabete)
print("diabete:", diabete)

print("Somme malade mda :", total_malade)
print("Somme normal mda :", total_normal)

print("\ntotal malade: ", total_g + diabete  + total_malade )


pour hyamd certaines data sont absentes par exemple 002456631_D_2 est dans le csv mais n'existe pas

In [ ]:
import pandas as pd
from pathlib import Path
import random

random.seed(42)

# -----------------------
# ROOTS
# -----------------------
root = Path("data")
mda_root = root / "mda"
glaucome_root = root / "glaucome"
dr_root = root / "diabete" / "diabetic-retinopathy-detection"

exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".gif"}

# -----------------------
# UTILS
# -----------------------
def list_images(folder: Path):
    folder = Path(folder)
    if not folder.exists():
        print("MANQUANT:", folder)
        return []
    imgs = [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]
    print("OK:", folder, "->", len(imgs))
    return imgs

def uniq_existing(paths):
    seen = set()
    out = []
    for p in paths:
        p = Path(p)
        if not p.exists():
            continue
        rp = p.resolve()
        if rp in seen:
            continue
        seen.add(rp)
        out.append(rp)
    return out

# =========================================================
# 1) GLAUCOME (glaucoma uniquement)
# =========================================================
glaucome = []

# ACRIMA
tmp = list_images(glaucome_root / "ACRIMA")
glaucome += [p for p in tmp if "glaucoma" in [x.lower() for x in p.parts]]

glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Testing" / "glaucoma")
glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Testing" / "glaucoma")
glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Testing" / "glaucoma")
glaucome += list_images(glaucome_root / "RIM-ONE" / "RIM-ONE" / "NOT PARTITIONED" / "glaucoma")

glaucome = uniq_existing(glaucome)
print("GLAUCOME uniques :", len(glaucome))

# =========================================================
# 2) MDA / AMD 
# =========================================================
mda = []

# ---------- AMDNet23 ----------
mda += list_images(mda_root / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Dataset" / "train" / "amd")
mda += list_images(mda_root / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Dataset" / "valid" / "amd")

# ---------- Fundus ----------
mda += list_images(mda_root / "Fundus" / "Fundus" / "ARMD")

# ---------- fundus ----------
mda += list_images(mda_root / "fundus-dataset" / "14. Age Related Macular Degeneration")

# -------- HYAMD --------
hyamd_root = Path(
    "data/mda/hyamd-high-resolution-fundus-image-dataset"
    "/hyamd-high-resolution-fundus-image-dataset-for-age-related-macular-degeneration-amd-diagnosis-1.0.0"
)

labels_hyamd = hyamd_root / "labels" / "labels.csv"
hyamd_images = hyamd_root / "Images"

df_hy = pd.read_csv(labels_hyamd)
hy_amd = df_hy[df_hy["AMD"] > 0]["image_id"].astype(str).tolist()

# index des fichiers par stem normalisé
files = [p for p in hyamd_images.iterdir() if p.is_file()]
index = {}
for p in files:
    key = p.stem.rstrip("_")
    index.setdefault(key, []).append(p)

hyamd_paths = []
missing = 0
for img_id in hy_amd:
    if img_id in index:
        hyamd_paths.extend(index[img_id])
    else:
        missing += 1

# dedup
hyamd_paths = list({p.resolve() for p in hyamd_paths})
# Après construction de index + hy_amd
missing_ids = [img_id for img_id in hy_amd if img_id not in index]
print("Manquants:", len(missing_ids))
print(missing_ids[:20])

print("HYAMD AMD attendues:", len(hy_amd))
print("HYAMD AMD ajoutées:", len(hyamd_paths))
print("HYAMD manquantes (labels sans fichier):", missing)

mda += hyamd_paths

# ---------- ODIR ----------
labels_odir = mda_root / "ODIR-5K"  / "full_df.csv"
df_odir = pd.read_csv(labels_odir)

odir_img_dir = mda_root / "ODIR-5K" / "ODIR-5K" / "Training Images"  # ajuste si besoin
odir_amd_ids = df_odir[df_odir["labels"].astype(str).str.contains(r"\bA\b", regex=True)]["ID"].tolist()

odir_paths = []
for id_ in odir_amd_ids:
    odir_paths += [odir_img_dir / f"{id_}_left.jpg", odir_img_dir / f"{id_}_right.jpg"]

odir_paths = [p for p in odir_paths if p.exists()]
print("ODIR AMD:", len(odir_paths))

mda += odir_paths

mda = uniq_existing(mda)


# =========================================================
# 3) DIABÈTE (2500)
# =========================================================
import pandas as pd
from pathlib import Path
import random

random.seed(42)

labels_csv = Path("data/diabete/trainLabels.csv/trainLabels.csv")
images_dir = Path("data/diabete/train.zip/train")



df = pd.read_csv(labels_csv)


# Construire un lookup: image(stem) -> level (int)
label_map = dict(zip(df["image"].astype(str), df["level"].astype(int)))

# Extensions images
exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".gif"}

# Lister les images réellement présentes (à plat)
imgs = [p for p in images_dir.iterdir() if p.is_file() and p.suffix.lower() in exts]

# Mélanger pour éviter un biais d'ordre
random.shuffle(imgs)

normal = []
diab = []
unknown = 0

TARGET = 1500

for p in imgs:
    stem = p.stem
    lvl = label_map.get(stem, None)

    if lvl is None:
        unknown += 1
        continue

    if lvl == 0:
        if len(normal) < TARGET:
            normal.append(p.resolve())
    else:
        if len(diab) < TARGET:
            diab.append(p.resolve())

    if len(normal) >= TARGET and len(diab) >= TARGET:
        break

print("Images dans le dossier :", len(imgs))
print("Sans label (non trouvées dans CSV):", unknown)
print("Normal sélectionnées :", len(normal))
print("Diabète sélectionnées :", len(diab))
print("TOTAL sélection :", len(normal) + len(diab))

In [ ]:
normaux_mda = []

normaux_mda += list_images(mda_root / "fundus-dataset" / "00. Normal")
normaux_mda += list_images(mda_root / "Fundus" / "Fundus" / "NORMAL")

# HYAMD normaux
hy_norm = df_hy[df_hy["AMD"] == 0]["image_id"].astype(str).tolist()
normaux_mda += [
    p for img_id in hy_norm
    for p in index.get(img_id.rstrip("_"), [])
]

# ODIR normaux (non A)
odir_norm_ids = df_odir[
    ~df_odir["labels"].astype(str).str.contains(r"\bA\b", regex=True)
]["ID"].tolist()

for id_ in odir_norm_ids:
    normaux_mda += [
        odir_img_dir / f"{id_}_left.jpg",
        odir_img_dir / f"{id_}_right.jpg"
    ]

normaux_mda = uniq_existing(normaux_mda)
random.shuffle(normaux_mda)


In [ ]:
normaux_glaucome = []
tmp = list_images(glaucome_root / "ACRIMA")
normaux_glaucome += [p for p in tmp if "normal" in [x.lower() for x in p.parts]]
normaux_glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Testing" / "normal")
normaux_glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Testing" / "normal")
normaux_glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Testing" / "normal")
normaux_glaucome += list_images(glaucome_root / "RIM-ONE" / "RIM-ONE" / "NOT PARTITIONED" / "normal")
normaux_glaucome = uniq_existing(normaux_glaucome)
random.shuffle(normaux_glaucome)


In [ ]:
normaux_diabete = []

norm_diab_ids = df[df["level"] == 0]["image"].astype(str).tolist()

normaux_diabete = [
    images_dir / f"{img_id}.jpeg"
    for img_id in norm_diab_ids
    if (images_dir / f"{img_id}.jpeg").exists()
]

normaux_diabete = uniq_existing(normaux_diabete)
random.shuffle(normaux_diabete)


In [ ]:
N_TOTAL = 2500
N_PER = N_TOTAL // 3

normaux = (
    normaux_mda[:N_PER] +
    normaux_glaucome[:N_PER] +
    normaux_diabete[:N_TOTAL - 2*N_PER]
)

random.shuffle(normaux)

print("NORMAUX MDA :", len(normaux_mda[:N_PER]))
print("NORMAUX GLAUCOME :", len(normaux_glaucome[:N_PER]))
print("NORMAUX DIABÈTE :", len(normaux_diabete[:N_TOTAL - 2*N_PER]))
print("NORMAUX TOTAL :", len(normaux))
paths_maladies = set(
    p.resolve()
    for p in (glaucome + mda + diab)
)

normaux = [p for p in normaux if p.resolve() not in paths_maladies]

print("NORMAUX après exclusion des maladies :", len(normaux))


il manque des normaux car duplication de path a cause du random

In [ ]:
N_TOTAL = 2500
manque = N_TOTAL - len(normaux)

if manque > 0:
    print(f"⚠️ Il manque {manque} normaux, complétion en cours...")

    # pool global de normaux possibles
    pool_normaux = (
        normaux_mda +
        normaux_glaucome +
        normaux_diabete
    )

    # nettoyage du pool
    pool_normaux = uniq_existing(pool_normaux)

    # retirer ceux déjà utilisés ou pathologiques
    used = set(p.resolve() for p in normaux)
    forbidden = set(p.resolve() for p in (glaucome + mda + diab))

    pool_normaux = [
        p for p in pool_normaux
        if p.resolve() not in used
        and p.resolve() not in forbidden
    ]

    random.shuffle(pool_normaux)

    normaux += pool_normaux[:manque]

print("NORMAUX FINAL :", len(normaux))


In [ ]:
# =========================================================
# 5) CSV FINAL
# =========================================================
rows = []
rows += [(str(p), "glaucome") for p in glaucome]
rows += [(str(p), "mda") for p in mda]
rows += [(str(p), "diabete") for p in diab]
rows += [(str(p), "normaux") for p in normaux]

df_out = pd.DataFrame(rows, columns=["path", "label"])
df_out["path"] = df_out["path"].apply(lambda p: str(Path(p).resolve()))
df_out = df_out.drop_duplicates(subset=["path"])

out = root / "dataset_selection.csv"
df_out.to_csv(out, index=False)

print("\nCSV écrit :", out)
print(df_out["label"].value_counts())
print("TOTAL :", len(df_out))

version sans image zoomer


In [ ]:
import pandas as pd
from pathlib import Path
import random

random.seed(42)

# -----------------------
# ROOTS
# -----------------------
root = Path("data")
mda_root = root / "mda"
glaucome_root = root / "glaucome"
dr_root = root / "diabete" / "diabetic-retinopathy-detection"

exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".gif"}

# -----------------------
# UTILS
# -----------------------
def list_images(folder: Path):
    folder = Path(folder)
    if not folder.exists():
        print("MANQUANT:", folder)
        return []
    imgs = [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]
    print("OK:", folder, "->", len(imgs))
    return imgs

def uniq_existing(paths):
    seen = set()
    out = []
    for p in paths:
        p = Path(p)
        if not p.exists():
            continue
        rp = p.resolve()
        if rp in seen:
            continue
        seen.add(rp)
        out.append(rp)
    return out

# =========================================================
# 1) GLAUCOME (glaucoma uniquement)
# =========================================================
glaucome = []

glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Testing" / "glaucoma")
glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Testing" / "glaucoma")
glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Training" / "glaucoma")
glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Testing" / "glaucoma")

glaucome = uniq_existing(glaucome)
print("GLAUCOME uniques :", len(glaucome))

# =========================================================
# 2) MDA / AMD 
# =========================================================
mda = []

# ---------- AMDNet23 ----------
mda += list_images(mda_root / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Dataset" / "train" / "amd")
mda += list_images(mda_root / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Fundus Image Dataset for  Age-Related Macular Degeneration Disease Detection" / "AMDNet23 Dataset" / "valid" / "amd")

# ---------- Fundus ----------
mda += list_images(mda_root / "Fundus" / "Fundus" / "ARMD")

# ---------- fundus ----------
mda += list_images(mda_root / "fundus-dataset" / "14. Age Related Macular Degeneration")

# -------- HYAMD --------
hyamd_root = Path(
    "data/mda/hyamd-high-resolution-fundus-image-dataset"
    "/hyamd-high-resolution-fundus-image-dataset-for-age-related-macular-degeneration-amd-diagnosis-1.0.0"
)

labels_hyamd = hyamd_root / "labels" / "labels.csv"
hyamd_images = hyamd_root / "Images"

df_hy = pd.read_csv(labels_hyamd)
hy_amd = df_hy[df_hy["AMD"] > 0]["image_id"].astype(str).tolist()

# index des fichiers par stem normalisé
files = [p for p in hyamd_images.iterdir() if p.is_file()]
index = {}
for p in files:
    key = p.stem.rstrip("_")
    index.setdefault(key, []).append(p)

hyamd_paths = []
missing = 0
for img_id in hy_amd:
    if img_id in index:
        hyamd_paths.extend(index[img_id])
    else:
        missing += 1

# dedup
hyamd_paths = list({p.resolve() for p in hyamd_paths})
# Après construction de index + hy_amd
missing_ids = [img_id for img_id in hy_amd if img_id not in index]
print("Manquants:", len(missing_ids))
print(missing_ids[:20])

print("HYAMD AMD attendues:", len(hy_amd))
print("HYAMD AMD ajoutées:", len(hyamd_paths))
print("HYAMD manquantes (labels sans fichier):", missing)

mda += hyamd_paths

# ---------- ODIR ----------
labels_odir = mda_root / "ODIR-5K"  / "full_df.csv"
df_odir = pd.read_csv(labels_odir)

odir_img_dir = mda_root / "ODIR-5K" / "ODIR-5K" / "Training Images"  # ajuste si besoin
odir_amd_ids = df_odir[df_odir["labels"].astype(str).str.contains(r"\bA\b", regex=True)]["ID"].tolist()

odir_paths = []
for id_ in odir_amd_ids:
    odir_paths += [odir_img_dir / f"{id_}_left.jpg", odir_img_dir / f"{id_}_right.jpg"]

odir_paths = [p for p in odir_paths if p.exists()]
print("ODIR AMD:", len(odir_paths))

mda += odir_paths

mda = uniq_existing(mda)


# =========================================================
# 3) DIABÈTE (2500)
# =========================================================
import pandas as pd
from pathlib import Path
import random

random.seed(42)

labels_csv = Path("data/diabete/trainLabels.csv/trainLabels.csv")
images_dir = Path("data/diabete/train.zip/train")



df = pd.read_csv(labels_csv)

# Construire un lookup: image(stem) -> level (int)
label_map = dict(zip(df["image"].astype(str), df["level"].astype(int)))

# Extensions images
exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".gif"}

# Lister les images réellement présentes (à plat)
imgs = [p for p in images_dir.iterdir() if p.is_file() and p.suffix.lower() in exts]

# Mélanger pour éviter un biais d'ordre
random.shuffle(imgs)

normal = []
diab = []
unknown = 0

TARGET = 1500

for p in imgs:
    stem = p.stem
    lvl = label_map.get(stem, None)

    if lvl is None:
        unknown += 1
        continue

    if lvl == 0:
        if len(normal) < TARGET:
            normal.append(p.resolve())
    else:
        if len(diab) < TARGET:
            diab.append(p.resolve())

    if len(normal) >= TARGET and len(diab) >= TARGET:
        break

print("Images dans le dossier :", len(imgs))
print("Sans label (non trouvées dans CSV):", unknown)
print("Normal sélectionnées :", len(normal))
print("Diabète sélectionnées :", len(diab))
print("TOTAL sélection :", len(normal) + len(diab))




In [ ]:
normaux_glaucome = []
normaux_glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "DRISHTI-GS" / "DRISHTI-GS" / "Testing" / "normal")
normaux_glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "LAG" / "LAG" / "Testing" / "normal")
normaux_glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Training" / "normal")
normaux_glaucome += list_images(glaucome_root / "ORIGA" / "ORIGA" / "Testing" / "normal")
normaux_glaucome = uniq_existing(normaux_glaucome)
random.shuffle(normaux_glaucome)


In [ ]:
N_TOTAL = 1500
N_PER = N_TOTAL // 3

normaux = (
    normaux_mda[:N_PER] +
    normaux_glaucome[:N_PER] +
    normaux_diabete[:N_TOTAL - 2*N_PER]
)

random.shuffle(normaux)

print("NORMAUX MDA :", len(normaux_mda[:N_PER]))
print("NORMAUX GLAUCOME :", len(normaux_glaucome[:N_PER]))
print("NORMAUX DIABÈTE :", len(normaux_diabete[:N_TOTAL - 2*N_PER]))
print("NORMAUX TOTAL :", len(normaux))
paths_maladies = set(
    p.resolve()
    for p in (glaucome + mda + diab)
)

normaux = [p for p in normaux if p.resolve() not in paths_maladies]

print("NORMAUX après exclusion des maladies :", len(normaux))


In [ ]:
# =========================================================
# 5) CSV FINAL
# ========================================================
random.shuffle(glaucome)
glaucome = glaucome[:1500]
rows = []
rows += [(str(p), "glaucome") for p in glaucome]
rows += [(str(p), "mda") for p in mda]
rows += [(str(p), "diabete") for p in diab]
rows += [(str(p), "normaux") for p in normaux]

df_out = pd.DataFrame(rows, columns=["path", "label"])
df_out["path"] = df_out["path"].apply(lambda p: str(Path(p).resolve()))
df_out = df_out.drop_duplicates(subset=["path"])

out = root / "dataset_selection_non_zoomer.csv"
df_out.to_csv(out, index=False)

print("\nCSV écrit :", out)
print(df_out["label"].value_counts())
print("TOTAL :", len(df_out))